# Extract texts from a document

In [25]:
import pandas as pd
import numpy as np
import PIL
import pytesseract
import spacy
import cv2
import os
from pdf2image import convert_from_path
from glob import glob
import logging



## Logic for text extraction from a document
1. Open the file using cv2 - currently only PDF
2. Convert the document into images
3. Parse the images to generate a dataframe of document details
4. Clean the dataframe to remove NAs and garbage words
5. Store the result of the dataframe into CSVs

In [26]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

pdfPath = r"../data/Binder3.pdf"

DEBUG:root:test


In [27]:
# this will put all the content of the data frame into a csv
def writecsv(df: pd.DataFrame, path: str):
    os.makedirs('csvs', exist_ok=True)
    df.to_csv(path)

In [28]:
def generateBoundedImage(image, index:int, df:pd.DataFrame):
    ## Draw lines on the text in the document
    level = 'word'
    for l, x, y, w, h, c, t in df[['level', 'left', 'top', 'width', 'height', 'conf', 'text']].values:
        if level == 'page':
            if l == 1:
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)
            else:
                continue

        elif level == 'block':
            if l == 2:
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            else:
                continue
        elif level == 'para':
            if l == 3:
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            else:
                continue
        elif level == 'line':
            if l == 4:
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
            else:
                continue
        elif level == 'word':
            if l == 5:
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(image, t, (x, y), cv2.FONT_HERSHEY_PLAIN,
                            1, (255, 0, 0), 2)
            else:
                continue
    
    boundedImagePath = "../data/bounded/boundingbox_"+ str(index)+".jpeg"
    cv2.imwrite(boundedImagePath, image)

In [37]:
# Generate metadata of a PDF file
def getFileData(filename: str):
    pages = convert_from_path(filename, 350)
    i = 1
    for page in pages:
        image_name = "../data/temp/Page_" + str(i) + ".jpg"
        page.save(image_name, "JPEG")
        i = i+1

    imagePathList = glob("../data/temp/*.jpg")

    #logging.info("total number of pages "+ str(len(imagePathList)))
    imagePathList.sort()

    finalDF = pd.DataFrame()

    col_int = ['level', 'page_num', 'block_num', 'par_num',
                'line_num', 'word_num', 'left', 'top', 'width', 'height']

    for index, path in enumerate   (imagePathList):
        if index == 1:
            break

        img = cv2.imread(path)
        data = pytesseract.image_to_data(img, lang='mar+eng')
        dataList = list(map(lambda x: x.split('\t'), data.split('\n')))
        df = pd.DataFrame(dataList[1:], columns=dataList[0])
        df.to_csv(index=False)

        ## Data cleaning
        df.dropna(inplace=True)  # drop the missing in rows
        df[col_int] = df[col_int].astype(int)
        df['conf'] = df['conf'].astype(float).astype(int)

        ## Draw lines on the text in the document
        #image = img.copy()
        #generateBoundedImage(image, index, df)
    
        finalDF = pd.concat([finalDF, df], ignore_index = True)

    # write df to csv
    writecsv(finalDF, "csvs/extracted.csv")
    
    dfWords=finalDF[finalDF['level'] == 5]

    writecsv(dfWords, "csvs/extractedText.csv")

    ## Filter dataframe to just words
    finalDF80 = dfWords[dfWords['conf'] > 80]
    #finalDF80 = finalDF80[["text"]]

    #finalDF80.dropna(inplace=True)

    return finalDF80


In [38]:
df80 = getFileData(pdfPath)
writecsv(df80, "csvs/output_80.csv")


In [31]:
dfArray = df80['text'].to_numpy()

dfArray

array(['  ', 'मराठी', 'विकिपीडिया', '=.', 'विकिपीडियन्स', 'कडून',
       '**आपले', 'हार्दिक', 'स्वागत', 'आहे.**', '      ', ' ', ' ', ' ',
       '  ', '  ', ' ', ' ', ' ', ' ', ' ', ' ', '   ', ' ', ' ', '  ',
       'जगाचे', 'स्वप्न', 'प्रत्यक्षात', 'आणा', 'कि', 'जगातील', 'त्या',
       'प्रत्येक', 'भाषेतील', 'ज्ञान,', 'माहिती', 'आणि', 'संवाद',
       'मराठीत', 'आणि', 'मराठीतील', 'माहीती,', 'ज्ञान', 'आणि', 'संबाद',
       'जगातील', 'प्रत्येक', 'भाषेत', 'तत्काळ', 'आणि', 'मुक्त', 'पणे',
       'उपलब्ध', 'चला', 'आपण', 'सारे', 'यामध्ये', 'वाक्यातील', 'काही',
       'निवडक', 'जबाबदाऱ्या', 'पेलण्यास', 'सक्षम', 'मराठी', 'विकिपीडिया',
       'चळवळीत', 'सहभागी', 'होऊ', 'या!', 'जर', 'हाडाचे', 'मराठी',
       'प्रेमी', 'असाल', 'तर', 'लागा', 'कामाला', 'आत्ता', 'पासून', 'या',
       'क्षणा', 'पासून', 'माय', 'मराठीस', 'द्या', 'मिळवून', 'तिच्या',
       'हक्काची', 'जागा', 'जी', 'अढळ', 'असेल', 'जो', 'पर्यंत', 'हे',
       'अंतराळ', 'आहे', 'आणि', 'त्यावर', 'जीवन', 'आहे', 'तो', 'पर्यंत',
       'वाटेत

In [34]:
dfArrayWithoutSpaces = [ele for ele in dfArray if ele.strip()]
largeString = " ".join(dfArrayWithoutSpaces)

largeString


"मराठी विकिपीडिया =. विकिपीडियन्स कडून **आपले हार्दिक स्वागत आहे.** जगाचे स्वप्न प्रत्यक्षात आणा कि जगातील त्या प्रत्येक भाषेतील ज्ञान, माहिती आणि संवाद मराठीत आणि मराठीतील माहीती, ज्ञान आणि संबाद जगातील प्रत्येक भाषेत तत्काळ आणि मुक्त पणे उपलब्ध चला आपण सारे यामध्ये वाक्यातील काही निवडक जबाबदाऱ्या पेलण्यास सक्षम मराठी विकिपीडिया चळवळीत सहभागी होऊ या! जर हाडाचे मराठी प्रेमी असाल तर लागा कामाला आत्ता पासून या क्षणा पासून माय मराठीस द्या मिळवून तिच्या हक्काची जागा जी अढळ असेल जो पर्यंत हे अंतराळ आहे आणि त्यावर जीवन आहे तो पर्यंत वाटेतील सारे अडथळे केवळ मानसिक असतील आणि ते पार करण्याचे सामर्थ्य मराठी भाषा तुम्हा आम्हा सर्वांना सदोदीत देवो ही विकिपीडिया परिचय : विकिपीडिया हा विकिपीडिया वाचकांनीच एकत्रितपणे इंटरनेट वर संपादित केलेला मुक्त ज्ञानकोश आहे. हे विकी आहे याचा अर्थ कुणीही अगदी तुम्हीसुध्दा ह्या वेबपेजची बहुतेक पाने किंवा लेख 'संपादन' असे लिहीलेल्या टॅबच्या चौकटीवर क्लिक करुन संपादनास सुरुवात करु शकता. संपादन म्हणजे नवे लेखन आणि जुन्या लेखनात इंग्रजी विकिपीडिया १५ जानेवारी २००१ ला स

In [ ]:
interestedCols = ['page_num', 'left', 'top', 'width', 'height', 'conf']
